In [ ]:
! pip install geopandas
! pip install fiona
! pip install folium
! pip install mapclassify
! pip install contextily
! pip install sentinelhub
! pip install rasterio

     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 16.7 MB 426 kB/s 
     |████████████████████████████████| 6.3 MB 40.8 MB/s 
     |████████████████████████████████| 19.3 MB 6.4 MB/s 
     |████████████████████████████████| 204 kB 4.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 132 kB 43.9 MB/s 
     |████████████████████████████████| 47 kB 4.6 MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-3.5.0-py3-none-any.whl size=222866 sha256=4c55ebcbfad509c5b5c0b4cd6703b00b6f78b95ac516e46c28860440570b6fc9
  Stored in directory: /root/.cache/pip/wheels/0d/fc/ed/52e82aa94e9e455b80e15a85d105703611c0fd951134e53364
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=704ceb41b0cd5c54b4e4021586f7b5e232046fb3ee7f619b373f5f0984d9ec5e
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4f

### Load info on positive data downloads

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ML-Climate-Predicting-Wildfires/src')

Mounted at /content/drive


In [65]:
import json 

fire_image_dir = "data/TrainingData/Fire-Positive-5mo/"
download_log_file = "{}download_log.json".format(fire_image_dir)

with open(download_log_file) as f:
  selected_image_info = json.load(f)

len(selected_image_info)

4576

In [66]:
downloaded_positive_samples = [k for k in selected_image_info if selected_image_info[k]]
len(downloaded_positive_samples)

2769

In [7]:
! pip install geopandas
! pip install fiona
! pip install folium
! pip install mapclassify
! pip install contextily
! pip install sentinelhub

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.3 MB 47.6 MB/s 
     |████████████████████████████████| 16.7 MB 36.4 MB/s 
     |████████████████████████████████| 19.3 MB 399 kB/s 
     |████████████████████████████████| 204 kB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 132 kB 64.2 MB/s 
     |████████████████████████████████| 47 kB 5.2 MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-3.5.0-py3-none-any.whl size=222866 sha256=0f9dc190c70a28919fe86ba3736a8b985cf66fc0249cb30c5b093fce1ce27a00
  Stored in directory: /root/.cache/pip/wheels/0d/fc/ed/52e82aa94e9e455b80e15a85d105703611c0fd951134e53364
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=145affa9fe971c3e5fba1034193c39fbcf7ece0e52ee2e49b7b69a887e985386
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4

In [8]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ML-Climate-Predicting-Wildfires/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### From fire boundary shapes, identify rectangular regions to target for Sentinel images

In [78]:
import fiona 
import geopandas as gpd
import pandas as pd
import contextily as cx
import datetime

# https://data-nifc.opendata.arcgis.com/datasets/nifc::wfigs-wildland-fire-locations-full-history/explore?location=-0.208721%2C121.952724%2C0.74
path_to_data = "data/RawData/NIFC_Wildland_Fire_Perimeters.gdb"
fires_gdf = gpd.read_file(path_to_data)
fires_gdf

,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,...,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,geometry
0,None,CERTIFIED,50.64,2020-08-06T23:13:07+00:00,2020-08-06T23:13:24+00:00,50.6,20.0,MTMCC,NaN,None,...,None,None,2020-08-06T19:50:29+00:00,2020-08-12T20:46:01+00:00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,None,None,POINT (-104.45751 45.78504)
1,None,DEFAULT,NaN,None,None,NaN,0.1,CALACC,NaN,None,...,None,None,2020-02-28T20:52:36+00:00,2020-02-28T20:52:36+00:00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,None,None,POINT (-118.18071 33.80898)
2,None,DEFAULT,NaN,2017-10-18T00:30:00+00:00,2017-10-18T00:35:00+00:00,50.0,50.0,MTKIC,NaN,None,...,None,None,2017-10-18T13:46:40+00:00,2017-11-09T22:08:19+00:00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,None,None,POINT (-114.83541 48.07395)
3,None,DEFAULT,NaN,None,None,NaN,NaN,CAMVIC,NaN,None,...,None,None,2019-07-01T20:10:12+00:00,2019-07-01T20:10:12+00:00,IRWIN,{5DF06F41-9948-49D3-B00A-2D3A1D1049C5},NaN,None,None,POINT (-117.15390 33.17639)
4,None,DEFAULT,NaN,None,None,NaN,NaN,None,NaN,None,...,None,None,2016-06-20T22:39:02+00:00,2016-06-20T22:39:02+00:00,IRWIN,{F378818E-D541-4E0A-9A44-C81886C2B8B4},NaN,None,None,POINT (-121.10418 38.83473)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212233,None,DEFAULT,NaN,None,None,NaN,NaN,SDGPC,NaN,None,...,None,None,2022-03-21T13:24:39+00:00,2022-03-21T13:24:39+00:00,IRWIN,{C15A3B09-A767-40FB-8404-DFA2CBC0F42F},0.0,None,None,POINT (-99.45001 44.06140)
212234,None,DEFAULT,NaN,None,None,1.0,1.0,ILILC,NaN,None,...,None,None,2022-03-21T13:29:12+00:00,2022-03-21T13:46:28+00:00,IRWIN,{054AFEB9-7BAF-425E-B528-2E6C436547D0},0.0,None,None,POINT (-88.03056 41.37154)
212235,None,DEFAULT,NaN,None,None,1.0,1.0,ILILC,NaN,None,...,None,None,2022-03-21T13:36:34+00:00,2022-03-21T13:48:06+00:00,IRWIN,{9F4846CC-07EC-4D84-9D37-DD016F11A502},0.0,None,None,POINT (-88.15883 41.34809)
212236,None,DEFAULT,NaN,None,None,1.0,1.0,ILILC,NaN,None,...,None,None,2022-03-21T13:39:35+00:00,2022-03-21T13:48:28+00:00,IRWIN,{A2794C09-838A-4434-97F4-9E82289DDC18},0.0,None,None,POINT (-88.16037 41.35492)


In [79]:
fires_gdf['discovery_date'] = pd.to_datetime(fires_gdf['FireDiscoveryDateTime']).dt.tz_localize(None)
fires_gdf = fires_gdf[fires_gdf['discovery_date'] > datetime.datetime(year=2017,month=1,day=1)]

In [80]:
fires_gdf = fires_gdf.to_crs('NAD83')
proj_gdf = fires_gdf.to_crs('+proj=cea')
centroid = proj_gdf.centroid.to_crs('+proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ')
buffer = centroid.buffer(1000, cap_style=3)
fires_gdf['buffer_4326'] = buffer.to_crs('EPSG:4326')

In [81]:
columns =  ['discovery_date',
            'IncidentName',
            'FireCause',
            'CalculatedAcres',
            'ContainmentDateTime',
            'UniqueFireIdentifier',
            'buffer_4326']
fires_df = fires_gdf[columns]

In [82]:
big_fires_gdf = fires_gdf[fires_gdf['DailyAcres']>100]
len(fires_gdf), len(big_fires_gdf)

(159446, 12225)

### Download Sentinel images for positive training points

In [54]:
from sentinelhub import SHConfig, SentinelHubCatalog, SentinelHubDownloadClient

config = SHConfig()
#stoner
config.sh_client_id = 'db3185e7-7d29-447b-91e6-0f5e31b71e42'
config.sh_client_secret = 'yM@G{l+)G4pN8>!u1h&R@.]jrP#%i3wa98G]}o>k'
config.save()
catalog = SentinelHubCatalog(config=config)
catalog.get_info()
client = SentinelHubDownloadClient(config=config)

In [55]:
%matplotlib inline

import datetime as dt

import numpy as np
import matplotlib.pyplot as plt

from sentinelhub import SHConfig, BBox, CRS, DataCollection,\
    SentinelHubRequest, filter_times, bbox_to_dimensions, \
    MimeType, SentinelHubDownloadClient


In [98]:
from datetime import timedelta, datetime

def get_bbox(fire):
    return fire['buffer_4326'].bounds

def get_time_interval(fire, t_pre, t_post):
    date_f = fire['discovery_date']
    delta_pre = timedelta(days=t_pre)
    delta_post = timedelta(days=t_post)
    date_i = date_f - delta_pre
    datestr_i = datetime.strftime(date_i, "%Y-%m-%d")
    date_f = date_f + delta_post
    datestr_f = datetime.strftime(date_f, "%Y-%m-%d")

    return [datestr_i, datestr_f]

evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12"],
                units: "DN"
            }],
            output: {
                bands: 12,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B11,
                sample.B12];
    }
"""

def get_percent_missing(image):
    return len(image[image==0]) / len(image[image>=0])

def get_best_timestamp(fire_bbox, fire_time_interval):
    results = []
    unique_acquisitions = None
    # Find valid dates in catalog
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=fire_bbox,
        time=fire_time_interval,
        query={
            "eo:cloud_cover": {
                "lt": 5
            }
        },
        fields={
            "include": [
                "id",
                "properties.datetime",
                "properties.eo:cloud_cover"
            ],
            "exclude": []
        }
    )
    results = list(search_iterator)
    if results:
      cloud_covers = [r['properties']['eo:cloud_cover'] for r in results]
      timestamps = search_iterator.get_timestamps()
      results_info = list(zip(timestamps, cloud_covers))
      results_info.sort(key = lambda x: x[1]) 
      time_difference = dt.timedelta(seconds=0)
      for timestamp, cc in results_info:
          request = SentinelHubRequest(
              evalscript=evalscript_all_bands,
              input_data=[
                  SentinelHubRequest.input_data(
                      data_collection=DataCollection.SENTINEL2_L2A,
                      time_interval=(timestamp - time_difference, timestamp + time_difference)
                  )
              ],
              responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
              bbox=fire_bbox,
              size=bbox_to_dimensions(fire_bbox, 20),
              config=config
          )
          data = request.get_data()
          if get_percent_missing(data[0]) < 0.2:
            return timestamp, data[0]
    return None, None

In [22]:
# https://hub.arcgis.com/datasets/1612d351695b467eba75fdf82c10884f/explore
path_to_data = "data/RawData/US_State_Boundaries"
us_states = gpd.read_file(path_to_data)
conus_states = us_states[(us_states['DAY_ADM']>0) & ~(us_states['STATE_ABBR'].isin(['HI', 'AK']))]

In [25]:
conus_boundary = conus_states.dissolve()
conus_boundary = conus_boundary['geometry'][0]

In [91]:
from shapely.geometry import Point
import numpy as np

def get_random_bbox_US():
    within = False
    while not within:
        x = np.random.uniform(conus_boundary.bounds[0], conus_boundary.bounds[2])
        y = np.random.uniform(conus_boundary.bounds[1], conus_boundary.bounds[3])
        within = conus_boundary.contains(Point(x, y))
    point = gpd.GeoSeries([Point(x,y)], crs=conus_states.crs)
    proj_point = point.to_crs('+proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ')
    buffer = proj_point.buffer(1000, cap_style=3)
    proj_buffer = buffer.to_crs('EPSG:4326')
    proj_point = point.to_crs('EPSG:4326')
    return proj_point, proj_buffer

# random = [get_random_bbox_US()[0] for _ in range(10)]

In [67]:
downloaded_fires_gdf = big_fires_gdf[big_fires_gdf['UniqueFireIdentifier'].isin(downloaded_positive_samples)]
len(downloaded_fires_gdf)

2769

In [88]:
fires_gdf_proj = fires_gdf.to_crs('+proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ')

In [94]:
def nearest_fire_occurence(point, date):
    recent_date_min, recent_date_max = (date-timedelta(days=30),
                                        date+timedelta(days=10))
    recent_fires = fires_gdf_proj[(fires_gdf_proj['discovery_date']>=recent_date_min)&(fires_gdf_proj['discovery_date']<=recent_date_max)]
    min_dist = float('inf')
    point_proj = point.to_crs('+proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ')
    for i, recent_fire in recent_fires.iterrows():
        fire_geom = recent_fire['geometry']
        dist = fire_geom.distance(point_proj[0])  
        if dist < min_dist:
            min_dist = dist
    return min_dist

In [113]:
import json 

fire_image_dir = "data/TrainingData/Fire-Negative-5mo/"
download_log_file = "{}download_log.json".format(fire_image_dir)

if os.path.exists(download_log_file):
  with open(download_log_file) as f:
    selected_image_info = json.load(f)
else:
  selected_image_info = {}

len(selected_image_info)

0

In [ ]:
twoweek_days = 14
random_sample_points = []
for index, fire in downloaded_fires_gdf.iterrows():
    fire_id = fire['UniqueFireIdentifier']
    print(index/max(big_fires_gdf.index)*100, '%')
    if not fire_id in selected_image_info:
      valid_timestamps = 0
      while not valid_timestamps >= 3:
          valid_sample = False
          while not valid_sample:
            point, bbox = get_random_bbox_US()
            nearest_fire_dist = nearest_fire_occurence(point, fire['discovery_date'])
            valid_sample = nearest_fire_dist > 2*1e4
          random_sample_points.append(point)
          bbox = BBox(bbox[0], crs=CRS.WGS84)
          fire_time_interval = get_time_interval(fire, t_pre=3*twoweek_days, t_post=-2*twoweek_days)
          first_timestamp, first_image = get_best_timestamp(bbox, fire_time_interval)
          if first_timestamp:
            timestamps = [first_timestamp]
            images = [first_image]
            valid_timestamps = 1
            for i in range(1,6):
              fire_time_interval = get_time_interval(fire, t_pre=(i+3)*twoweek_days, t_post=-(i+2)*twoweek_days)
              timestamp, image = get_best_timestamp(bbox, fire_time_interval)
              valid_timestamps += 1 if timestamp else 0
              timestamps.append(timestamp if timestamp else timestamps[-1])
              images.append(image if image is not None else images[-1])

      selected_image_info[fire_id] = [(point[0].x, point[0].y), [str(timestamp.date()) for timestamp in timestamps]]
      for i, image in enumerate(images):
          print(fire['discovery_date'], '-->', timestamps[i])
          # True color
          # plt.figure(figsize=(9,9))
          # plt.imshow(np.clip(image[:,:,1:4]*.5/1000, 0, 255))
          # plt.show()
          with open(fire_image_dir + "{}_{}.npy".format(fire_id, i), 'wb') as f:
              np.save(f, image)
      with open(download_log_file, 'w') as f: 
          f.write(json.dumps(selected_image_info))

0.002827028274994464 %
2019-10-26 21:29:00 --> 2019-09-25 17:34:28+00:00
2019-10-26 21:29:00 --> 2019-09-25 17:34:28+00:00
2019-10-26 21:29:00 --> 2019-08-26 17:34:33+00:00
2019-10-26 21:29:00 --> 2019-08-16 17:34:34+00:00
2019-10-26 21:29:00 --> 2019-08-16 17:34:34+00:00
2019-10-26 21:29:00 --> 2019-07-17 17:34:37+00:00
0.00848108482498339 %
2021-01-28 21:37:24 --> 2020-12-21 18:33:19+00:00
2021-01-28 21:37:24 --> 2020-12-21 18:33:19+00:00
2021-01-28 21:37:24 --> 2020-11-21 18:33:23+00:00
2021-01-28 21:37:24 --> 2020-11-11 18:33:24+00:00
2021-01-28 21:37:24 --> 2020-11-01 18:33:26+00:00
2021-01-28 21:37:24 --> 2020-10-17 18:33:24+00:00
0.02261622619995571 %
2020-06-15 21:29:00 --> 2020-05-18 17:04:12+00:00
2020-06-15 21:29:00 --> 2020-05-18 17:04:12+00:00
2020-06-15 21:29:00 --> 2020-04-18 17:04:07+00:00
2020-06-15 21:29:00 --> 2020-04-18 17:04:07+00:00
2020-06-15 21:29:00 --> 2020-04-18 17:04:07+00:00
2020-06-15 21:29:00 --> 2020-02-28 17:04:01+00:00
0.024500911716618688 %
2020-07-21

In [ ]:
fire_hazards_list = []
random_sample_list_x = []
random_sample_list_y = []
closest_fire_dist_list = []
for unique_id in fires_gdf['UniqueFireIdentifier']:
    if unique_id in fires_wfpi:
        fire_hazard, coord, closest_fire_dist = fires_wfpi[unique_id]
        fire_hazards_list.append(fire_hazard)
        closest_fire_dist_list.append(closest_fire_dist)
        random_sample_list_x.append(coord[0])
        random_sample_list_y.append(coord[1])
    else:
        fire_hazards_list.append(None)
        closest_fire_dist_list.append(None)
        random_sample_list_x.append(None)
        random_sample_list_y.append(None)
fires_gdf['WFPI'] = fire_hazards_list
fires_gdf['ClosestFireDist'] = closest_fire_dist_list
fires_gdf['RandomSampleXCoord'] = random_sample_list_x
fires_gdf['RandomSampleYCoord'] = random_sample_list_y

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
random_samples_data = list(zip(fires_gdf['UniqueFireIdentifier'], fires_gdf['discovery_date'], fire_hazards_list, random_sample_list_x, random_sample_list_y))
random_samples_df = pd.DataFrame(random_samples_data, columns=['CorrespondingUniqueFireIdentifier', 'discovery_date', 'WFPI', 'XCoord', 'YCoord'])

In [ ]:
random_samples_df.to_csv('data/ProcessedData/RandomSamplePoints.csv')